## Imports

In [1]:
# %matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from os import path
import metos3d.petsc_mod as petsc
import data_conversion as dc
import ipywidgets as widgets

## Load data

In [2]:
data_path = r"D:\Bachelorarbeit Data\0000\save"
data = np.load(path.join(data_path, "data.npy"))
a_k = np.abs(np.load(path.join(data_path, "a_k.npy")))
b_k = np.abs(np.load(path.join(data_path, "b_k.npy")))
land_sea_mask = petsc.read_PETSc_matrix('metos3d\\landSeaMask.petsc')
n1, n2 = np.shape(land_sea_mask.T)
n3 = int(np.amax(land_sea_mask))

In [3]:
n1,n2,n3

(128, 64, 15)

## Convert 2D-Data to 4D

The data is in a 3D grid, which represents the position of each measure point in space. Each point is eaither `None` if there is no measuring point in the posiotion of a numpy array with the data associated to this point.

In [4]:
ak_3d = dc.convert_2d_to_4d(a_k, land_sea_mask)
bk_3d = dc.convert_2d_to_4d(b_k, land_sea_mask)
data_3d = dc.convert_2d_to_4d(data, land_sea_mask)


## Definition of useful functions

In [5]:
def extract_a_0(X):
    if not X is None:
        return X[0]
    else:
        return np.nan


extract_a_0 = np.vectorize(extract_a_0, otypes="f")

a0_3d = extract_a_0(ak_3d)

In [6]:
def get_max_coef(ak, bk):
    """Find the maximal coefficient for each 0 with 1 if it is a_k of 2 if it is b_k and also the index, meaning the k of a_k or b_k"""
    if ak is None or bk is None:
        return np.nan, 0, np.nan
    else:
        max_a = np.amax(ak[1:])
        max_a_idx = np.argmax(ak[1:]) +1
        max_b = np.amax(bk)
        max_b_idx = np.argmax(bk)
        if max_a >= max_b:
            return max_a, 1, max_a_idx
        else:
            return max_b, 2, max_b_idx


get_max_coef = np.vectorize(get_max_coef)

max_coef, coef_type, coef_idx = get_max_coef(ak_3d, bk_3d)

max_rel = max_coef/ a0_3d

In [7]:
@widgets.interact(layer=(0, 14, 1), data_type=["max", "type", "ratio", "idx", "a0"])
def plot_3d(layer=0, data_type="ratio"):
    if data_type == "max":
        data = max_coef
    elif data_type == "ratio":
        data = max_rel
    elif data_type == "type":
        data = coef_type
    elif data_type == "idx":
        data = coef_idx
    elif data_type == "a0":
        data = a0_3d

    fig, ax = plt.subplots(figsize=(15, 7))
    long, lat = np.meshgrid(np.linspace(-90, 90, n2),
                            np.linspace(0, 360, n1))
    data = data[:, :, layer]
    print("max:", np.nanmax(data))
    print("min:", np.nanmin(data))
    plot = ax.contourf(lat, long, data)
    fig.colorbar(plot, ax=ax)
    plt.draw()

interactive(children=(IntSlider(value=0, description='layer', max=14), Dropdown(description='data_type', index…

In [8]:
import pandas as pd
pd.set_option('display.max_rows', 100)

In [9]:
def filter_nan(X):
    return X[X==X]

def filter_zero(X):
    return X[X != 0]

In [10]:
@widgets.interact(layer=(0,14,1), sort_by=["coef", "type", "idx","ratio","a0"], ascending=[True, False], num=(10,100,1), threshold=(0,10,0.1))
def show_data(layer=0, sort_by="ratio", ascending=False, num=10, threshold=0):
    data = {"coef": filter_nan(max_coef[:,:,layer]), "type": filter_zero(coef_type[:,:,layer]), "idx": filter_nan(coef_idx[:,:,layer]), "ratio": filter_nan(max_rel[:,:,layer]), "a0": filter_nan(a0_3d[:,:,layer])}
    df = pd.DataFrame(data)
    df = df[df[sort_by] >= threshold]
    print(len(df))
    return df.sort_values(sort_by, ascending=ascending).head(num)

interactive(children=(IntSlider(value=0, description='layer', max=14), Dropdown(description='sort_by', index=3…

In [11]:
layers = [i + 1 for i in range(15)]
count = [len(filter_nan(max_coef[:, :, i])) for i in range(15)]
median_max_coef = [np.median(filter_nan(max_coef[:, :, i])) for i in range(15)]
mean_max_coef = [np.mean(filter_nan(max_coef[:, :, i])) for i in range(15)]
layer_max_coef = [np.max(filter_nan(max_coef[:, :, i])) for i in range(15)]
mean_a0 = [np.mean(filter_nan(a0_3d[:, :, i])) for i in range(15)]
median_a0 = [np.median(filter_nan(a0_3d[:, :, i])) for i in range(15)]
max_a0 =  [np.max(filter_nan(a0_3d[:, :, i])) for i in range(15)]
median_ratio = [np.median(filter_nan(max_rel[:, :, i])) for i in range(15)]
mean_ratio = [np.mean(filter_nan(max_rel[:, :, i])) for i in range(15)]
max_ratio = [np.max(filter_nan(max_rel[:, :, i])) for i in range(15)]
pd.DataFrame(data={"layers": layers,"number of measured points":count,   "median_max_coef": median_max_coef, "mean_max_coef": mean_max_coef, "max_coef": layer_max_coef,
             "mean_a0": mean_a0, "median_a0": median_a0, "max a0": max_a0,  "median of max ratio": median_ratio, "mean of max ratio" : mean_ratio, "max of ratio" : max_ratio})


,layers,number of measured points,median_max_coef,mean_max_coef,max_coef,mean_a0,median_a0,max a0,median of max ratio,mean of max ratio,max of ratio
0,1,4448,0.042989,0.049926,0.395649,0.602630,0.385723,2.557166,0.099161,0.148976,1.788243
1,2,4392,0.019485,0.026941,0.475646,0.749656,0.524623,3.771885,0.045531,0.067965,1.748717
2,3,4334,0.016373,0.020504,0.236272,1.051745,0.897501,4.642633,0.019743,0.030966,0.434879
3,4,4259,0.008547,0.011998,0.129664,1.301029,1.203156,4.902974,0.008122,0.013137,0.251607
4,5,4164,0.004719,0.006876,0.068551,1.557096,1.507724,4.768317,0.003501,0.005638,0.110538
5,6,4088,0.002864,0.004233,0.056974,1.806533,1.791551,4.261612,0.001671,0.002745,0.064204
6,7,4026,0.001667,0.002826,0.036461,2.041114,2.031863,4.087929,0.000849,0.001472,0.025237
7,8,3940,0.001026,0.001702,0.042294,2.203353,2.176857,4.389729,0.000496,0.000803,0.015286
8,9,3822,0.000621,0.000953,0.025517,2.267723,2.252358,5.016776,0.000286,0.000476,0.007051
9,10,3679,0.000394,0.000614,0.008514,2.286396,2.322809,5.710725,0.000169,0.000306,0.006123


In [12]:
def get_ratios(data, a0):
    if data is None or a0 is None:
        return None

    return data / a0

get_ratios = np.vectorize(get_ratios)
a_ratios = get_ratios(ak_3d, a0_3d)
b_ratios = get_ratios(bk_3d, a0_3d)

def ratio_count_over(ratios, threshold):
    if ratios is None:
        return 0
    else:
        return len(ratios[ratios >= threshold])
ratio_count_over = np.vectorize(ratio_count_over)

In [13]:
@widgets.interact(layer=(0,14,1), threshold=(0,1,0.05))
def values_above_ratio(layer=0, threshold=0):
    fig, ax = plt.subplots(figsize=(15, 7))
    long, lat = np.meshgrid(np.linspace(-90, 90, n2),
                            np.linspace(0, 360, n1))
    ak_count = ratio_count_over(a_ratios[:,:,layer],threshold )
    bk_count = ratio_count_over(b_ratios[:,:,layer], threshold)
    count = ak_count + bk_count
    print("max:", np.max(count))
    plot = ax.contourf(lat, long, count)
    fig.colorbar(plot, ax=ax)
    plt.draw()

interactive(children=(IntSlider(value=0, description='layer', max=14), FloatSlider(value=0.0, description='thr…

In [14]:
dev = data.std(axis=1)

In [15]:
dev_3, n1,n2,n3 = petsc.reshape_vector_to_3d(land_sea_mask, dev)
long, lat = np.meshgrid(np.linspace(-90, 90, n2),
                        np.linspace(0, 360, n1))

In [16]:
@widgets.interact(layer=(0,14,1))
def std_dev(layer=0,):
    fig, ax = plt.subplots(figsize=(15, 7))
    print("max:", np.nanmax(dev_3[:,:,layer]))
    print("min:", np.nanmin(dev_3[:,:,layer]))
    plot = ax.contourf(lat, long, dev_3[:,:,layer])
    fig.colorbar(plot, ax=ax)
    plt.draw()

interactive(children=(IntSlider(value=0, description='layer', max=14), Output()), _dom_classes=('widget-intera…